In [1]:
import os

In [2]:
!cd /content
!rm -rf /content/mt

In [3]:
# clone if does not exist
if not os.path.isdir('/content/mt'):
    !git clone https://github.com/fbelderink/mt.git
%cd /content/mt
!git pull
!git checkout fb/dev
!git pull

Cloning into 'mt'...
remote: Enumerating objects: 755, done.
remote: Counting objects: 100% (755/755), done.
remote: Compressing objects: 100% (356/356), done.
remote: Total 755 (delta 380), reused 733 (delta 366), pack-reused 0
Receiving objects: 100% (755/755), 3.40 MiB | 12.26 MiB/s, done.
Resolving deltas: 100% (380/380), done.
/content/mt
Already up to date.
Branch 'fb/dev' set up to track remote branch 'fb/dev' from 'origin'.
Switched to a new branch 'fb/dev'
Already up to date.


In [4]:
import training.train as train
from utils.ConfigLoader import ConfigLoader
from utils.hyperparameters import Hyperparameters
from google.colab import drive

In [10]:
drive_dir = '/content/drive'
drive.mount(drive_dir)

dataset_dir = os.path.join(drive_dir,'MyDrive/data/mt')

model_hyperparameters = Hyperparameters(
    ConfigLoader("configs/config.yaml").get_config())

train.train(os.path.join(dataset_dir,
                        f"train7k_w{model_hyperparameters.window_size}.pt"),
            os.path.join(dataset_dir,
                        f"val7k_w{model_hyperparameters.window_size}.pt"),
            model_hyperparameters,
            val_rate=200)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
training on cuda:0
Number of Batches: 2094
Batch Size: 200
total number of trainable parameters: 48735765
Layers:
source_embedding: Embedding(6797, 200)
target_embedding: Embedding(6461, 200)
fc_source: Linear(in_features=1800, out_features=300, bias=True)
fc_source_bn: BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
fc_target: Linear(in_features=800, out_features=300, bias=True)
fc_target_bn: BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
fc1: Linear(in_features=600, out_features=500, bias=True)
fc1_bn: BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
fc2: Linear(in_features=500, out_features=6461, bias=True)
fc2_bn: BatchNorm1d(6461, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
output_layer: Linear(in_features=6461, out_features=6461, bias=T

KeyboardInterrupt: 

In [7]:
from datetime import datetime
date = datetime.today().strftime('%d-%m-%Y')

In [ ]:
!zip -r {dataset_dir}/checkpoints.zip eval/checkpoints/{date}/

updating: eval/checkpoints/12-06-2024/ (stored 0%)
updating: eval/checkpoints/12-06-2024/20_13_43.pth


zip error: Interrupted (aborting)


In [11]:
import torch
from assignments.assignment4 import *
from utils.file_manipulation import load_data
from preprocessing.dictionary import Dictionary

data_de = load_data("data/raw/multi30k.dev.de")
data_en = load_data("data/raw/multi30k.dev.en")

dict_de = Dictionary.load(os.path.join(dataset_dir, "dicts/train_dict_de.pkl"))
dict_en = Dictionary.load(os.path.join(dataset_dir, "dicts/train_dict_en.pkl"))

model = torch.load(f"eval/checkpoints/{date}/21_44_38.pth")

test_model_bleu(model, data_de, data_en, dict_de, dict_en,
                3, model_hyperparameters.window_size, True, None)

Model BLEU: 0.21043451518637923
